In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy.optimize import root_scalar
from sklearn.linear_model import (LinearRegression, HuberRegressor,
                                  RANSACRegressor, TheilSenRegressor)
from bokeh.layouts import layout
from bokeh.models import Slider, CustomJS
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook

from tqdm import notebook

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
num = 749

In [4]:
def find_fiber(edges, **args):
    # global p1
    y, x = edges.shape
    num = np.tile(np.linspace(0, 1, y), (x, 1)).T
    if args.get('plot') == True:
        p1 = args.get('p1')
        p1.image(image=[edges], x=0, y=0, dw=1, dh=1, palette="Spectral11")
    edges = edges > 0
    s = np.sum(edges, axis=0)
    m = np.sum(edges * num, axis=0) / s
    st = np.sqrt(np.sum(edges * (num - m)**2, axis=0) / s)
    # print(st.shape)
    # print(st)
    std = np.nanmedian(st)
    # print(std)
    # if args.get('plot') == True:
    #     plt.plot(st, 'o')
    #     plt.plot([0, x - 1], [std, std])
    #     plt.show()
    X = np.linspace(0, 1, x)
    bad = np.argwhere(np.isnan(m))
    X = np.delete(X, bad)
    m = np.delete(m, bad)
    ransac = RANSACRegressor(random_state=42).fit(X.reshape([-1, 1]), m)
    if args.get('plot') == True:
        p2 = args.get('p2')
        pred = ransac.predict(X[[0, -1]].reshape([-1, 1]))
        p2.circle(X, m, size=5)
        p2.line([0, 1], pred, line_color='red')
        p2.line([0, 1], pred+std, line_color='green')
        p2.line([0, 1], pred-std, line_color='green')
    return float(ransac.estimator_.coef_), float(ransac.predict([[0]])), std


def turn_crop(im, alf, b, std, **args):
    w = args.get('width', 4)
    up = max(b, b + alf)
    down = min(b, b + alf)
    y, x = im.shape
    up = int(np.clip(up + w * std, 0, 1) * y)
    down = int(np.clip(down - w * std, 0, 1) * y)
    cr_im = im[down:up, :]
    y_new = cr_im.shape[0]
    M = cv2.getRotationMatrix2D([x / 2, y_new / 2],
                                np.arctan(alf * y / x) * 180 / np.pi, 1)
    rotated = cv2.warpAffine(cr_im,
                             M, [cr_im.shape[1], cr_im.shape[0]],
                             borderMode=cv2.BORDER_REPLICATE)
    y_min = int(y_new / 2 - w * std * y)
    y_max = int(y_new / 2 + w * std * y)
    rotated = rotated[y_min:y_max]
    if args.get('plot') == True:
        p3 = args.get('p3')
        p3.image(image=[rotated],
                 x=0,
                 y=0,
                 dw=1,
                 dh=rotated.shape[0] / y,
                 palette="Spectral11")
        # plt.imshow(rotated)
        # plt.show()
    return rotated


def cuts(m, n):
    y, x = m.shape
    step = x / n
    b = np.ceil(np.arange(step, x, step)).astype(int)
    lis = np.split(m, b, axis=1)
    means = []
    for l in lis:
        means.append(l.mean(axis=1))
    return np.stack(means)


def fit(m, x):
    ransac = RANSACRegressor(random_state=42).fit(x.reshape([-1, 1]), m)
    pred = ransac.predict(x.reshape([-1, 1]))
    std = np.mean((pred - m)**2)
    return std, {'x': x[[0, -1]], 'y': pred[[0, -1]]}


class window_fit():
    def __init__(self, mas, w):
        self.mas = mas
        self.w = w
        self.x, self.y = mas.shape
        self.v = np.ones_like(mas) * float('-inf')
        self.Y = np.arange(self.y)
        self.boards = np.zeros([self.x, 2])

    def get(self, x, y):

        y -= self.w / 2
        x0 = math.floor(x)
        x1 = math.ceil(x)
        y0 = math.floor(y)
        y1 = math.ceil(y)
        s0 = self.calc(x0, y0)
        s1 = self.calc(x1, y1)
        s = s1 * (y - y0) + s0 * (1 - (y - y0))
        return s

    def calc(self, x, y):
        if self.v[x, y] == float('-inf'):
            s, p = fit(self.mas[x, y:y + self.w], self.Y[y:y + self.w])
            self.v[x, y] = math.log(s)
            return math.log(s)
        else:
            return self.v[x, y]

    def get_betwin(self, y0, y1):
        y0, y1 = min(y0, y1), max(y0, y1)
        y0 = max(0, math.ceil(y0))
        y1 = min(self.y - 1, math.floor(y1))
        return np.arange(y0, y1 + 1)

    def get_mas(self, x):
        X = np.argwhere(self.v[x] != float('-inf'))
        Y = self.v[x, X]
        X = X + self.w / 2
        return X.flatten(), Y.flatten()

    def calc_gerd(self, n):
        Y = np.ceil(np.linspace(0, self.y - self.w - 1, n)).astype(int)
        for x in range(self.x):
            for y in Y:
                self.calc(x, y)

    def find_borders(self, kof):
        for x in range(self.x):
            p, val = self.get_mas(x)
            # plt.plot(p,val)
            # plt.show()
            ma = val.max()
            mi = val.min()
            tr = mi * kof + ma * (1 - kof)
            ind = np.argwhere(val > tr).flatten()
            min_ind = max(ind.min() - 1, 0)
            b = [min_ind, min_ind+1]
            tr_l = val[b].min()
            tr_h = val[b].max()
            if tr < tr_l: tr = tr_l
            if tr > tr_h: tr = tr_h
            y0 = root_scalar(lambda y: self.get(x, y) - tr,
                             method='brentq',
                             bracket=p[b]).root
            max_ind = min(ind.max() + 1, len(val)-1)
            b = [max_ind - 1, max_ind]
            tr_l = val[b].min()
            tr_h = val[b].max()
            if tr < tr_l: tr = tr_l
            if tr > tr_h: tr = tr_h
            y1 = root_scalar(lambda y: self.get(x, y) - tr,
                             method='brentq',
                             bracket=p[b]).root
            self.boards[x, :] = [y0 + self.w / 2 - 0.5, y1 - self.w / 2 - 0.5]

In [5]:
def count_foto(dir,path):
    p0 = figure(height=400, width=800)
    p1 = figure(height=300, width=400)
    p2 = figure(height=300, width=400)
    p3 = figure(height=220, width=800)
    p4 = figure(height=300, width=400)
    p5 = figure(height=300, width=400)
    p6 = figure(height=300, width=800)
    p7 = figure(height=220, width=800)
    p = layout([[p0], [p1, p2], [p3], [p4, p5], [p6], [p7]])

    im = cv2.imread(dir + path, cv2.IMREAD_GRAYSCALE)
    p0.image(image=[im], x=0, y=0, dw=1, dh=1, palette="Spectral11")
    y, x = im.shape
    kof = 20
    tr = 100
    im4 = cv2.resize(im, dsize=(x // kof, y // kof))
    edges = cv2.Canny(im4, tr, tr, L2gradient=True)
    alf, b, std = find_fiber(edges, plot=True, p1=p1, p2=p2)
    rotated = turn_crop(im, alf, b, std, width=4, plot=True, p3=p3)
    mas = cuts(rotated, 30)
    # p4.line(np.linspace(0,1,mas.shape[1]),mas[0])
    wf = window_fit(mas, 20)
    wf.calc_gerd(20)
    wf.find_borders(0.5)

    x = 20
    y0 = wf.boards[x, 0]
    y1 = wf.boards[x, 1]
    ins = wf.mas[x, wf.get_betwin(y0, y1)]
    ma = ins.max()
    mi = ins.min()
    vals = [mi, ma, ma, mi, mi]
    points = [y0, y0, y1, y1, y0]
    p4.line(points, vals, line_color='red')
    p4.circle(wf.Y, wf.mas[x], size=5)

    X, Y = wf.get_mas(x)
    vals = [wf.get(x, y0 - wf.w / 2 + 0.5), wf.get(x, y1 + wf.w / 2 + 0.5)]
    points = [y0 - wf.w / 2 + 0.5, y1 + wf.w / 2 + 0.5]
    vals = [wf.get(x, points[0]), wf.get(x, points[1])]
    p5.line(X, Y)
    p5.circle(X, Y)
    p5.line(points, vals, line_color='red')

    X = np.linspace(0, 1, wf.boards.shape[0])
    p6.line(X, wf.boards[:, 0])
    p6.line(X, wf.boards[:, 1])
    p7.line(X, wf.boards[:, 1] - wf.boards[:, 0])
    std1, point1 = fit(wf.boards[:, 0], X)
    std2, point2 = fit(wf.boards[:, 1], X)
    p6.line([0, 1], point1['y'], line_color='red')
    p6.line([0, 1], point2['y'], line_color='red')
    p7.line([0, 1], point2['y'] - point1['y'], line_color='red')
    resalt = np.mean(point2['y'] - point1['y'])
    
    return resalt, p

In [6]:
import os
import re

black_list = ['Scan.ipynb']
dir = f'testPic/{num}/'
files = os.listdir(dir)
files = [file for file in files if file not in black_list]
files = [{
    'name': file,
    'x': float(re.findall(r'-?\d+\.?\d*', file)[-1])
} for file in files]
files.sort(key=lambda a: a['x'])

In [7]:
plots = []
x = []
y = []
for file in notebook.tqdm(files[0:]):
    if True:
        print(file)
        x.append(file['x'])
        res, p = count_foto(dir, file['name'])
        y.append(res)
        plots.append(p)


  0%|          | 0/119 [00:00<?, ?it/s]

{'name': 'im749_0.5.jpg', 'x': 0.5}
{'name': 'im749_1.0.jpg', 'x': 1.0}
{'name': 'im749_1.5.jpg', 'x': 1.5}
{'name': 'im749_2.0.jpg', 'x': 2.0}
{'name': 'im749_2.5.jpg', 'x': 2.5}
{'name': 'im749_3.0.jpg', 'x': 3.0}
{'name': 'im749_3.5.jpg', 'x': 3.5}
{'name': 'im749_4.0.jpg', 'x': 4.0}
{'name': 'im749_4.5.jpg', 'x': 4.5}
{'name': 'im749_5.0.jpg', 'x': 5.0}
{'name': 'im749_5.5.jpg', 'x': 5.5}
{'name': 'im749_6.0.jpg', 'x': 6.0}
{'name': 'im749_6.5.jpg', 'x': 6.5}
{'name': 'im749_7.0.jpg', 'x': 7.0}
{'name': 'im749_7.5.jpg', 'x': 7.5}
{'name': 'im749_8.0.jpg', 'x': 8.0}
{'name': 'im749_8.5.jpg', 'x': 8.5}
{'name': 'im749_9.0.jpg', 'x': 9.0}
{'name': 'im749_9.5.jpg', 'x': 9.5}
{'name': 'im749_10.0.jpg', 'x': 10.0}
{'name': 'im749_10.5.jpg', 'x': 10.5}


C:\Users\ariva\AppData\Local\Temp\ipykernel_7636\2437975524.py:10: RuntimeWarning: invalid value encountered in divide
  m = np.sum(edges * num, axis=0) / s


{'name': 'im749_11.0.jpg', 'x': 11.0}


C:\Users\ariva\AppData\Local\Temp\ipykernel_7636\2437975524.py:10: RuntimeWarning: invalid value encountered in divide
  m = np.sum(edges * num, axis=0) / s


...

{'name': 'im749_51.5.jpg', 'x': 51.5}
{'name': 'im749_52.0.jpg', 'x': 52.0}


C:\Users\ariva\AppData\Local\Temp\ipykernel_7636\2437975524.py:10: RuntimeWarning: invalid value encountered in divide
  m = np.sum(edges * num, axis=0) / s


{'name': 'im749_52.5.jpg', 'x': 52.5}


C:\Users\ariva\AppData\Local\Temp\ipykernel_7636\2437975524.py:10: RuntimeWarning: invalid value encountered in divide
  m = np.sum(edges * num, axis=0) / s


{'name': 'im749_53.0.jpg', 'x': 53.0}
{'name': 'im749_53.5.jpg', 'x': 53.5}
{'name': 'im749_54.0.jpg', 'x': 54.0}
{'name': 'im749_54.5.jpg', 'x': 54.5}
{'name': 'im749_55.0.jpg', 'x': 55.0}
{'name': 'im749_55.5.jpg', 'x': 55.5}
{'name': 'im749_56.0.jpg', 'x': 56.0}
{'name': 'im749_56.5.jpg', 'x': 56.5}
{'name': 'im749_57.0.jpg', 'x': 57.0}
{'name': 'im749_57.5.jpg', 'x': 57.5}
{'name': 'im749_58.0.jpg', 'x': 58.0}
{'name': 'im749_58.5.jpg', 'x': 58.5}
{'name': 'im749_59.0.jpg', 'x': 59.0}
{'name': 'im749_59.5.jpg', 'x': 59.5}


In [8]:
sh = figure(height=600, width=800)
sh.line(x, np.array(y) * 1.82)#, size=15)
show(sh)

In [24]:
np.mean(y) * 1.82 /2 

39.13740702774939

In [9]:
path = f'testPic/res/{num}/'
output_file(path + 'shape.html')
save(sh)
for i, coord in enumerate(x):
    print(coord)
    output_file(path + 'pic' + str(int(coord*10)) + '.html')
    save(plots[i])

0.5
1.0
1.5
2.0
2.5
3.0
3.5
4.0
4.5
5.0
5.5
6.0
6.5
7.0
7.5
8.0
8.5
9.0
9.5
10.0
10.5
11.0
11.5
12.0
12.5
13.0
13.5
14.0
14.5
15.0
15.5
16.0
16.5
17.0
17.5
18.0
18.5
19.0
19.5
20.0
20.5
21.0
21.5
22.0
22.5
23.0
23.5
24.0
24.5
25.0
25.5
26.0
26.5
27.0
27.5
28.0
28.5
29.0
29.5
30.0
30.5
31.0
31.5
32.0
32.5
33.0
33.5
34.0
34.5
35.0
35.5
36.0
36.5
37.0
37.5
38.0
38.5
39.0
39.5
40.0
40.5
41.0
41.5
42.0
42.5
43.0
43.5
44.0
44.5
45.0
45.5
46.0
46.5
47.0
47.5
48.0
48.5
49.0
49.5
50.0
50.5
51.0
51.5
52.0
52.5
53.0
53.5
54.0
54.5
55.0
55.5
56.0
56.5
57.0
57.5
58.0
58.5
59.0
59.5
